# Research Notebook
## Felix Groh
## Date: 26.01.2025
# 1: Experience
## Describe at least one research activity you worked on this week. 
- Solved the two state Schrödinger equation for inelastic scattering.
- Derived formulas for determening phaseshift and crosssections from two state solutions

# 2: What? (What happened?)
## Solution to the two state Schrödinger equation

The original differential equation for the two state problem (See Reference [3])
\begin{equation}
\frac{1}{r^2} \frac{d}{dr} \left( r^2 \frac{d\mathbf{R}_\ell}{dr} \right)  + \left(k_1^2 - \frac{\ell (\ell+1)}{r^2} - 2\mu \mathbf{V}(r) \right) \mathbf{R}_\ell = 0 ,
\end{equation}
can be simplified by coordinate transform to the following form:
\begin{equation}
	\chi''_{\ell}(r)= \begin{pmatrix}\frac{\ell(\ell+1)}{x^{2}} - a^{2} & V(r) \\ V(r) & \frac{\ell(\ell+1)}{x^{2}} - c^{2}\end{pmatrix} \chi_{\ell}(r)
\end{equation}
Beneath is a, for the benefit of readability, simplified implementation for the two state solution, where $\eta$ is given by:
\begin{equation}
    \eta(x, \beta, \ell) = \frac{\ell (\ell + 1)}{x^2} - \beta,
\end{equation}
where $\beta \in \{a^2, c^2 \}$.

---
```python
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from enum import Enum
from scipy.integrate import odeint, solve_ivp
from scipy.special import spherical_jn, spherical_yn



def solve_Schrodinger_eqn(a, c, d, x_range, init_cond, L=0, rtol=1e-12, atol=1e-12, method='solve_ivp'):
    def eta(beta, t, L = 0):
        return L * (L + 1) / (t**2) - beta

    def V_Potential(t, b):
        return - np.exp(-t / b) / t

    def RHS(x, Z):
        Chi = Z[:4].reshape(2, 2) 
        V = Z[4:].reshape(2, 2)  # V = dChi/dt

        K = np.array([[eta(a, x, L), V_Potential(x, d)], [V_Potential(x, d), eta(c, x, L)]])

        dVdt = K @ V  # Matrix product K * F
        
        return np.concatenate(Xi.flatten(), dVdt.flatten())
    
    if method == 'solve_ivp':
        solution = solve_ivp(RHS, x_range, init_cond, rtol=rtol, atol=atol)
        
    else:
        raise Exception("Only methods 'solve_ivp' recognized")
    
    return solution
```
---

In order to solve the equation, we transform it in to a differential equation of first order:
\begin{equation}
\frac{d}{dx} \begin{bmatrix} \chi \\ V \end{bmatrix} = \begin{bmatrix} V \\ -k \cdot \chi \end{bmatrix}
\end{equation}

where:

\begin{equation}
\chi = \begin{bmatrix} \chi_{11} & \chi_{12} \\ \chi_{21} & \chi_{22} \end{bmatrix}, \quad V = \frac{d\chi}{dt} = \begin{bmatrix} v_{11} & v_{12} \\ v_{21} & v_{22} \end{bmatrix}
\end{equation}

and the full expanded system is:

\begin{equation}
\frac{d}{dt} \begin{bmatrix} \chi_{11} & \chi_{12} \\ \chi_{21} & \chi_{22} \\ v_{11} & v_{12} \\ v_{21} & v_{22} \end{bmatrix} =
\begin{bmatrix} v_{11} & v_{12} \\ v_{21} & v_{22} \\ -k_{11} \chi_{11} - k_{12} \chi_{21} & -k_{11} \chi_{12} - k_{12} \chi_{22} \\ -k_{21} \chi_{11} - k_{22} \chi_{21} & -k_{21} \chi_{12} - k_{22} \chi_{22} \end{bmatrix}
\end{equation}
$k$ now represents the original matrix, containing the coefficients:
\begin{equation}
 k = \begin{pmatrix}\frac{\ell(\ell+1)}{x^{2}} - a^{2} & V(r) \\ V(r) & \frac{\ell(\ell+1)}{x^{2}} - c^{2}\end{pmatrix}
\end{equation}
## 


# 3: So what? (What does it mean?)
## 
Checking results at this stage of development is not quite possible, since a check of the two state solution would require full a implementation of phaseshift and crosssection. For methods and implementation of computing the crossections see Reference [3] and for the one state solution [1].
# 4. Now what? (What's next?)
## Determening the coefficients required to calclating the phaseshift
In order to determine the coefficients we match the numerical solution to the Schrödinger equation to the asymptotic wavefunction. At large $r \ll 0, r = r_m$ the solution converges, and the potential becomes ineffective:  
\begin{equation}
\chi(r) \sim h_l^{(1)}(kr) C^{(1)} + h_l^{(2)}(kr) C^{(2)},
\end{equation}
where $C^{(1)}$ and $C^{(2)}$ are $2 \times 2$ matrices, containing the coefficients for the two state solution. The wavefunction as well as its derivatives can be written as: 
\begin{align}
\chi(r_m) &= h_l^{(1)}(kr_m) C^{(1)} + h_l^{(2)}(kr_m) C^{(2)}, \\
\chi'(r_m) &= \frac{d}{dr}\left[h_l^{(1)}(kr) C^{(1)} + h_l^{(2)}(kr) C^{(2)}\right]_{r=r_m}
\end{align}
The derivatives of the Hankel functions are:
\begin{align}
h_l^{(1)'}(kr) &= k \left[h_{l-1}^{(1)}(kr) - \frac{l+1}{kr} h_l^{(1)}(kr)\right], \\
h_l^{(2)'}(kr) &= k \left[h_{l-1}^{(2)}(kr) - \frac{l+1}{kr} h_l^{(2)}(kr)\right]
\end{align}
For $r = r_m$ this gives us two equations:
\begin{align}
\chi(r_m) &= h_l^{(1)}(kr_m) C^{(1)} + h_l^{(2)}(kr_m) C^{(2)} \\
\chi'(r_m) &= h_l^{(1)'}(kr_m) C^{(1)} + h_l^{(2)'}(kr_m) C^{(2)}
\end{align}
These can be rewritten as a linear system.
\begin{equation}
\begin{pmatrix}
h_l^{(1)}(kr_m) & h_l^{(2)}(kr_m) \\
h_l^{(1)'}(kr_m) & h_l^{(2)'}(kr_m)
\end{pmatrix} \cdot \begin{pmatrix}
    C^{(1)} \\
    C^{(2)}
\end{pmatrix} = \begin{pmatrix}
\chi(r_m) \\
\chi'(r_m)
\end{pmatrix}
\end{equation}
Now we can solve for $c^{(1)}$ and $c^{(2)} $:
\begin{equation} \begin{pmatrix}
    C^{(1)} \\
    C^{(2)}
\end{pmatrix} = \begin{pmatrix}
h_l^{(1)}(kr_m) & h_l^{(2)}(kr_m) \\
h_l^{(1)'}(kr_m) & h_l^{(2)'}(kr_m)
\end{pmatrix}^{-1} \begin{pmatrix}
\chi(r_m) \\
\chi'(r_m)
\end{pmatrix}
\end{equation}
# 5. Bibliography
[1] Blennow, M., Clementz, S., & Herrero-Garcia, J. (2017). Self-interacting inelastic dark matter: A viable solution to the small scale structure problems. Journal of Cosmology and Astroparticle Physics, 2017(03), 048.

[2] Tulin, S., Yu, H. B., & Zurek, K. M. (2013). Beyond collisionless dark matter: particle physics dynamics for dark matter halo structure. Physical Review D—Particles, Fields, Gravitation, and Cosmology, 87(11), 115007.

[3] Durand, L. (n.d.). InSIDious matter (S. Tulin, Supervisor). Department of Physics and Astronomy, York University, Canada.

<span style="color: red;">1. Add a section on motivation. What is the goal of the research? Why are you doing what you are doing. </span>
<span style="color: red;">2. Use more active voice instead of passive. For example "we started by with equation __ from [_] and coded a funtion to do X." </span>
<span style="color: red;">3. Use this section to describe how what you have done has added to your goal...</span>
<span style="color: red;">4. Ok, but tie this in to the overall goal. </span>

| Category       | Points      |
| ------------- |:------------:|
| Formatting    |       2       |
| Experience    |       1       |
| What?         |       1       |
| So what?      |       2       |
| Now what?     |       2       |
| Bibliography  |       3       |
| Style         |       3       |
| Total         |       15      |